In [2]:
# UCSD ECE 143 Programming for Data Analysis
# Winter 2023
# Author: m3cheung@ucsd.edu
# Final Project: create morbidity by race bar graph

Import modules

In [11]:
import pandas as pd
import plotly.express as px
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import sys

Specify year to analyze

In [12]:
year = 2021.0

Load csv files into Pandas DataFrames

In [16]:
morbidity_df = pd.read_csv(\
    "../data_csv/races-morbidity.csv").drop(columns=["Notes", "Maternal Morbidity Checked", "Maternal Morbidity Checked Code", "Mother's Hispanic Origin Code", "Year Code", "Mother's Single Race 6 Code"])
total_births_df = pd.read_csv(\
    "../data_csv/races-total_births.csv").drop(columns=["Notes", "Year Code", "Mother's Single Race 6 Code", "Mother's Hispanic Origin Code"])

display(morbidity_df)
display(total_births_df)


,Mother's Single Race 6,Year,Mother's Hispanic Origin,Births
0,American Indian or Alaska Native,2018.0,Hispanic or Latino,103.0
1,American Indian or Alaska Native,2018.0,Not Hispanic or Latino,575.0
2,American Indian or Alaska Native,2019.0,Hispanic or Latino,114.0
3,American Indian or Alaska Native,2019.0,Not Hispanic or Latino,571.0
4,American Indian or Alaska Native,2020.0,Hispanic or Latino,103.0
...,...,...,...,...
86,NaN,NaN,NaN,NaN
87,NaN,NaN,NaN,NaN
88,NaN,NaN,NaN,NaN
89,NaN,NaN,NaN,NaN


,Mother's Single Race 6,Mother's Hispanic Origin,Year,Births
0,American Indian or Alaska Native,Hispanic or Latino,2018.0,7004.0
1,American Indian or Alaska Native,Hispanic or Latino,2019.0,7522.0
2,American Indian or Alaska Native,Hispanic or Latino,2020.0,7950.0
3,American Indian or Alaska Native,Hispanic or Latino,2021.0,8331.0
4,American Indian or Alaska Native,Hispanic or Latino,NaN,30807.0
...,...,...,...,...
119,NaN,NaN,NaN,NaN
120,NaN,NaN,NaN,NaN
121,NaN,NaN,NaN,NaN
122,NaN,NaN,NaN,NaN


Clean up data

In [19]:
# remove NaN rows
morbidity_df = morbidity_df.dropna()
total_births_df = total_births_df.dropna()

# reset indices to start from 0
total_births_df = total_births_df.reset_index(drop=True)
morbidity_df = morbidity_df.reset_index(drop=True)

# create new df for the ratio!
ratio_df = total_births_df.merge(morbidity_df, on=["Year","Mother's Single Race 6", "Mother's Hispanic Origin"], how="left")

# rename columns
ratio_df = ratio_df.rename(columns={"Births_x":"Total Births", "Births_y":"Births w/ Morbidity"})

# remove NaN rows
ratio_df = ratio_df.dropna()

# display(ratio_df)

# isolate hispanic origin rows
hisp_df = ratio_df[ratio_df["Mother's Hispanic Origin"] == "Hispanic or Latino"]
ratio_df = ratio_df[ratio_df["Mother's Hispanic Origin"] == "Not Hispanic or Latino"]

# # find sum of Births and Morbidity columns by Marital Status and Edu Lvl
# hisp_df["Mother\'s Single Race 6"] = "Hispanic Origin"
# hisp_df = hisp_df.groupby(["Mother's Education", "Marital Status"]).aggregate({"Mother's Single Race 6":"first", "Year":"first", "Total Births": "sum", "Births w/ Morbidity": "sum"}).reset_index()

# # add rows to include hispanic origin for year
# ratio_df = pd.concat([ratio_df, hisp_df]).drop(columns="Mother's Hispanic Origin").reset_index(drop=True)


display(ratio_df)

,Mother's Single Race 6,Mother's Hispanic Origin,Year,Total Births,Births w/ Morbidity
4,American Indian or Alaska Native,Not Hispanic or Latino,2018.0,29092.0,575.0
5,American Indian or Alaska Native,Not Hispanic or Latino,2019.0,28450.0,571.0
6,American Indian or Alaska Native,Not Hispanic or Latino,2020.0,26813.0,499.0
7,American Indian or Alaska Native,Not Hispanic or Latino,2021.0,26124.0,606.0
16,Asian,Not Hispanic or Latino,2018.0,240798.0,5517.0
17,Asian,Not Hispanic or Latino,2019.0,238769.0,5623.0
18,Asian,Not Hispanic or Latino,2020.0,219068.0,5291.0
19,Asian,Not Hispanic or Latino,2021.0,213813.0,5190.0
28,Black or African American,Not Hispanic or Latino,2018.0,552029.0,6543.0
29,Black or African American,Not Hispanic or Latino,2019.0,548075.0,6659.0


Calculate Ratio for each row (The numbers end up being so high though!)

In [23]:
# create column for MMR = (Deaths/Live Births * 100,000)
ratio_df["Morbidity per 100,000 live births"] = ratio_df["Births w/ Morbidity"] / ratio_df["Total Births"] * 10000

display(ratio_df)

,Mother's Single Race 6,Mother's Hispanic Origin,Year,Total Births,Births w/ Morbidity,"Maternal Morbidity Ratio per 100,000 Live Births","Morbidity per 100,000 live births"
4,American Indian or Alaska Native,Not Hispanic or Latino,2018.0,29092.0,575.0,1.976488,197.648838
5,American Indian or Alaska Native,Not Hispanic or Latino,2019.0,28450.0,571.0,2.007030,200.702988
6,American Indian or Alaska Native,Not Hispanic or Latino,2020.0,26813.0,499.0,1.861038,186.103756
7,American Indian or Alaska Native,Not Hispanic or Latino,2021.0,26124.0,606.0,2.319706,231.970602
16,Asian,Not Hispanic or Latino,2018.0,240798.0,5517.0,2.291132,229.113199
17,Asian,Not Hispanic or Latino,2019.0,238769.0,5623.0,2.354996,235.499583
18,Asian,Not Hispanic or Latino,2020.0,219068.0,5291.0,2.415232,241.523180
19,Asian,Not Hispanic or Latino,2021.0,213813.0,5190.0,2.427355,242.735474
28,Black or African American,Not Hispanic or Latino,2018.0,552029.0,6543.0,1.185264,118.526382
29,Black or African American,Not Hispanic or Latino,2019.0,548075.0,6659.0,1.214980,121.497970


Do it by percentage instead?

In [8]:
ratio_df["Percentage of Live Births in Population"] =  ratio_df["Births w/ Morbidity"] / ratio_df["Total Births"] * 100

display(ratio_df)

,Mother's Education,Marital Status,Mother's Single Race 6,Year,Total Births,Births w/ Morbidity,"Maternal Morbidity Ratio per 100,000 Live Births",Percentage of Live Births in Population
0,8th grade or less,Married,Asian,2021.0,2890.0,51.0,1765.0,1.764706
1,8th grade or less,Married,Black or African American,2021.0,3030.0,64.0,2112.0,2.112211
2,8th grade or less,Married,White,2021.0,19023.0,239.0,1256.0,1.256374
3,8th grade or less,Unmarried,Black or African American,2021.0,2343.0,44.0,1878.0,1.877934
4,8th grade or less,Unmarried,White,2021.0,3966.0,46.0,1160.0,1.159859
...,...,...,...,...,...,...,...,...
85,High school graduate or GED completed,Unmarried,Hispanic Origin,2021.0,123379.0,1330.0,1078.0,1.077979
86,"Master's degree (MA, MS, MEng, MEd, MSW, MBA)",Married,Hispanic Origin,2021.0,20196.0,291.0,1441.0,1.440879
87,"Master's degree (MA, MS, MEng, MEd, MSW, MBA)",Unmarried,Hispanic Origin,2021.0,3826.0,50.0,1307.0,1.306848
88,"Some college credit, but not a degree",Married,Hispanic Origin,2021.0,56166.0,640.0,1139.0,1.139479


Create Map

In [10]:
fig = px.histogram(ratio_df[ratio_df["Marital Status"] == "Unmarried"], x="Mother's Education", y="Percentage of Live Births in Population", barmode="group", color="Mother's Single Race 6", title="", height=800,
             width=1000, category_orders={"Single Race 6": ["More than one race", "Asian", "Hispanic Origin", "White", "Black or African American", "American Indian or Alaska Native"]})
fig.show()
